In [10]:
# import packages
import pandas as pd
import csv
from sklearn.impute import KNNImputer

In [4]:
# load data: is "6.1_weather_parameters-2.csv" the latest version?

merged_df = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/6.1_weather_parameters-2.csv")

print(merged_df.head(), "\n")
print(merged_df.info())

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-01         6.0     17.8375                 15.0        20.0   
1  2013-07-01         6.0     17.8375                 15.0        20.0   
2  2013-07-01         6.0     17.8375                 15.0        20.0   
3  2013-07-01         6.0     17.8375                 15.0        20.0   
4  2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  Niederschlag  
0          0.0          1.0  148.828353        0.0         1.0           0.3  
1          0.0          2.0  535.856285        0.0         1.0           0.3  
2          0.0          3.0  201.198426        0.0         1.0           0.3  
3          0.0          4.0   65.890169        0.0         1.0           0.3  
4          0.0          5.0  317.475875        0.0         1.0           0.3   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data 

### Impute missing values

We do not know why values are missing in weather code data (and also the other datasets we use) so we assume it is missing completely at random (MCAR).
A listwise deletion would cost us too many datapoints (e.g. for "Wettercode" we have 9334 dates but only 7009 with valid weather code. If we delete all dates with missing datapoints, this would cost us roughly 20% of our information and that is a lot!)
So we decided to fill the data. Since the weathercode is categorial and not numeric, we cannot use a method based on distance or statistical metrics like average.
We chose to use KNN (K-nearest neighbour) method because we have other columns like cloudiness and precipitation and temperature and windspeed which might have to do with the weathercode and could help fill the weathercode through similarities.

But first we need to fill the missing values from the other columns which have less missing points. In the case of cloudiness we use the mode imputation which fills the missing values with the most common value.

For temperature and windspeed and as well precipitation (although not sure if it is good for precipitation - we are not meteorologists) we use a linear interpolation. This means that a missing value is filled with the average of the last non-missing value and the next. 

In [5]:
# mode imputation for cloudiness

# Function to create a mask column for imputed values
def create_imputation_mask(df, column_name):
    mask_name = f'imp_mask_{column_name}'
    mask = df[column_name].isna().astype(int)  # 1 for missing values, 0 for original values
    return mask_name, mask

# Create mask for 'Bewoelkung'
bew_mask_name, bew_mask = create_imputation_mask(merged_df, 'Bewoelkung')

# Find the mode of the 'cloudiness' column
mode_cloudiness = merged_df['Bewoelkung'].mode()[0]

# Impute missing values with the mode
merged_df['Bewoelkung'].fillna(mode_cloudiness, inplace=True)

# Add mask column to the dataframe
merged_df[bew_mask_name] = bew_mask

print(f"Mode of 'Bewoelkung'{mode_cloudiness}")
print(merged_df['Bewoelkung'].head(15))

print(merged_df.info("Bewoelkung")) # check if all values are non-null


Mode of 'Bewoelkung'7.0
0     6.0
1     6.0
2     6.0
3     6.0
4     6.0
5     7.0
6     7.0
7     7.0
8     7.0
9     7.0
10    7.0
11    7.0
12    7.0
13    7.0
14    7.0
Name: Bewoelkung, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datum                9334 non-null   object 
 1   Bewoelkung           9334 non-null   float64
 2   Temperatur           9318 non-null   float64
 3   Windgeschwindigkeit  9318 non-null   float64
 4   Wettercode           7009 non-null   float64
 5   KielerWoche          9334 non-null   float64
 6   Warengruppe          9334 non-null   float64
 7   Umsatz               9334 non-null   float64
 8   Feiertage            9334 non-null   float64
 9   Ferientage           9334 non-null   float64
 10  Niederschlag         9334 non-null   float64
 11  imp_mask_Bewoelkung  9334 non-n

/tmp/ipykernel_14990/2989050151.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Bewoelkung'].fillna(mode_cloudiness, inplace=True)


In [6]:
# linear interpolation for windspeed, temperature and precipitation

# Function to create a mask column for interpolated values
def create_interpolation_mask(df, column_name):
    mask_name = f'imp_mask_{column_name}'
    mask = df[column_name].isna().astype(int)  # 1 for missing values, 0 for original values
    return mask_name, mask

# Create masks for each column
temp_mask_name, temp_mask = create_interpolation_mask(merged_df, 'Temperatur')
wind_mask_name, wind_mask = create_interpolation_mask(merged_df, 'Windgeschwindigkeit')
precip_mask_name, precip_mask = create_interpolation_mask(merged_df, 'Niederschlag')

# Interpolate missing values for Precipitation and WindSpeed
merged_df['Temperatur'] = merged_df['Temperatur'].interpolate(method='linear')
merged_df['Windgeschwindigkeit'] = merged_df['Windgeschwindigkeit'].interpolate(method='linear')
merged_df['Niederschlag'] = merged_df['Niederschlag'].interpolate(method='linear')

# Fill any remaining NaNs (e.g., at the beginning or end) using forward fill
merged_df['Temperatur'].fillna(method='bfill', inplace=True)
merged_df['Windgeschwindigkeit'].fillna(method='bfill', inplace=True)
merged_df['Niederschlag'].fillna(method='bfill', inplace=True)

# Add mask columns to the dataframe
merged_df[temp_mask_name] = temp_mask
merged_df[wind_mask_name] = wind_mask
merged_df[precip_mask_name] = precip_mask

# Display the DataFrame after interpolation
print(merged_df.info()) # check if all values are non-null


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Datum                         9334 non-null   object 
 1   Bewoelkung                    9334 non-null   float64
 2   Temperatur                    9334 non-null   float64
 3   Windgeschwindigkeit           9334 non-null   float64
 4   Wettercode                    7009 non-null   float64
 5   KielerWoche                   9334 non-null   float64
 6   Warengruppe                   9334 non-null   float64
 7   Umsatz                        9334 non-null   float64
 8   Feiertage                     9334 non-null   float64
 9   Ferientage                    9334 non-null   float64
 10  Niederschlag                  9334 non-null   float64
 11  imp_mask_Bewoelkung           9334 non-null   int64  
 12  imp_mask_Temperatur           9334 non-null   int64  
 13  imp

/tmp/ipykernel_14990/2039611570.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Temperatur'].fillna(method='bfill', inplace=True)
/tmp/ipykernel_14990/2039611570.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['Temperatur'].fillna(method='bfill', inplace=True)
/tmp/ipykernel_14990/2039611570.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using a

In [18]:
# KNN imputation for Wettercode

# Keep the 'Datum' column aside
datum = merged_df['Datum']

# Select relevant columns for imputation
columns_for_imputation = ['Wettercode', 'Windgeschwindigkeit', 'Niederschlag', 'Bewoelkung', 'Temperatur']

# Subset the dataframe to include only the relevant columns
df_impute = merged_df[columns_for_imputation]

# Store the original Wettercode values
original_wettercode = df_impute['Wettercode']
missing_mask = original_wettercode.isna().astype(int)  # 1 for missing values, 0 for original values

# Encode the 'Wettercode' column if necessary (KNNImputer can work with categorical data as long as they are numerical)
# If 'Wettercode' is already numerical, skip this step
df_impute['Wettercode'] = df_impute['Wettercode'].astype('category').cat.codes

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Apply the KNN imputer to the dataframe
df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_impute), columns=columns_for_imputation)

# Convert imputed 'Wettercode' back to integers and ensure it's within the valid categorical range
df_imputed['Wettercode'] = df_imputed['Wettercode'].round().astype(int)
df_imputed['Wettercode'] = df_imputed['Wettercode'].clip(0, 99)

# If you encoded 'Wettercode' earlier, convert it back to the original form
#df_imputed['Wettercode'] = df_imputed['Wettercode'].round().astype(int)  # If 'Wettercode' is supposed to be integers

# Convert imputed values back to categorical codes
#df_imputed[:, 0] = original_wettercode.combine_first(pd.Series(df_imputed[:, 0])).astype(int)

# Convert imputed 'Wettercode' back to integers (optional, if needed)
#df_imputed[:, 0] = df_imputed[:, 0].round().astype(int)


# Ensure only the NaN values in 'Wettercode' are imputed
df_imputed['Wettercode'] = original_wettercode.combine_first(df_imputed['Wettercode'])

# Add the 'Datum' column back to the imputed dataframe
df_imputed['Datum'] = datum

# Merge the imputed 'Wettercode' back into the original dataframe
df_imp = merged_df.copy()
df_imp['Wettercode'] = df_imputed['Wettercode']
# Add the mask column to indicate imputed values
df_imp['imp_mask_Wettercode'] = missing_mask


# Display the imputed dataframe
print(df_imp.info())
print("\n", df_imp.head())
print("\n", df_imp['Wettercode'].describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Datum                         9334 non-null   object 
 1   Bewoelkung                    9334 non-null   float64
 2   Temperatur                    9334 non-null   float64
 3   Windgeschwindigkeit           9334 non-null   float64
 4   Wettercode                    9334 non-null   float64
 5   KielerWoche                   9334 non-null   float64
 6   Warengruppe                   9334 non-null   float64
 7   Umsatz                        9334 non-null   float64
 8   Feiertage                     9334 non-null   float64
 9   Ferientage                    9334 non-null   float64
 10  Niederschlag                  9334 non-null   float64
 11  imp_mask_Bewoelkung           9334 non-null   int64  
 12  imp_mask_Temperatur           9334 non-null   int64  
 13  imp

/tmp/ipykernel_14990/2542910797.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_impute['Wettercode'] = df_impute['Wettercode'].astype('category').cat.codes


In [ ]:
# next try...
from category_encoders import OrdinalEncoder

# KNN imputation for Wettercode

# Keep the 'Datum' column aside
datum = merged_df['Datum']

# Select relevant columns for imputation
columns_for_imputation = ['Wettercode', 'Windgeschwindigkeit', 'Niederschlag', 'Bewoelkung', 'Temperatur']

# Subset the dataframe to include only the relevant columns
df_impute = merged_df[columns_for_imputation]

# Store the original Wettercode values
original_wettercode = df_impute['Wettercode']
missing_mask = original_wettercode.isna().astype(int)  # 1 for missing values, 0 for original values

# Encode the categorical 'Wettercode' column
ordinal_encoder = OrdinalEncoder()
encoded_df = ordinal_encoder.fit_transform(merged_df[['Wettercode']])

# Combine the encoded 'Wettercode' with the rest of the DataFrame
df_encoded = merged_df.copy()
df_encoded['Wettercode'] = encoded_df[:, 0]

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=2)

# Apply the KNN imputer to the dataframe
imputed_df_encoded = knn_imputer.fit_transform(df_encoded)
df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_impute), columns=columns_for_imputation)

# Convert imputed 'Wettercode' back to integers and ensure it's within the valid categorical range
df_imputed['Wettercode'] = df_imputed['Wettercode'].round().astype(int)
df_imputed['Wettercode'] = df_imputed['Wettercode'].clip(0, 99)

# If you encoded 'Wettercode' earlier, convert it back to the original form
#df_imputed['Wettercode'] = df_imputed['Wettercode'].round().astype(int)  # If 'Wettercode' is supposed to be integers

# Convert imputed values back to categorical codes
#df_imputed[:, 0] = original_wettercode.combine_first(pd.Series(df_imputed[:, 0])).astype(int)

# Convert imputed 'Wettercode' back to integers (optional, if needed)
#df_imputed[:, 0] = df_imputed[:, 0].round().astype(int)


# Ensure only the NaN values in 'Wettercode' are imputed
df_imputed['Wettercode'] = original_wettercode.combine_first(df_imputed['Wettercode'])

# Add the 'Datum' column back to the imputed dataframe
df_imputed['Datum'] = datum

# Merge the imputed 'Wettercode' back into the original dataframe
df_imp = merged_df.copy()
df_imp['Wettercode'] = df_imputed['Wettercode']
# Add the mask column to indicate imputed values
df_imp['imp_mask_Wettercode'] = missing_mask


# Display the imputed dataframe
print(df_imp.info())
print("\n", df_imp.head())
print("\n", df_imp['Wettercode'].describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Datum                         9334 non-null   object 
 1   Bewoelkung                    9334 non-null   float64
 2   Temperatur                    9334 non-null   float64
 3   Windgeschwindigkeit           9334 non-null   float64
 4   Wettercode                    9334 non-null   float64
 5   KielerWoche                   9334 non-null   float64
 6   Warengruppe                   9334 non-null   float64
 7   Umsatz                        9334 non-null   float64
 8   Feiertage                     9334 non-null   float64
 9   Ferientage                    9334 non-null   float64
 10  Niederschlag                  9334 non-null   float64
 11  imp_mask_Bewoelkung           9334 non-null   int64  
 12  imp_mask_Temperatur           9334 non-null   int64  
 13  imp

/tmp/ipykernel_14990/2542910797.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_impute['Wettercode'] = df_impute['Wettercode'].astype('category').cat.codes


In [21]:
#and again...

import pandas as pd
from sklearn.impute import KNNImputer
from category_encoders import OrdinalEncoder


# Define the columns to be used for KNN imputation
columns_for_knn = ['Windgeschwindigkeit', 'Niederschlag', 'Bewoelkung', 'Temperatur']

# Encode the categorical 'Wettercode' column
ordinal_encoder = OrdinalEncoder()
encoded_df = ordinal_encoder.fit_transform(merged_df[['Wettercode']])

# Combine the encoded 'Wettercode' with the rest of the DataFrame
df_encoded = merged_df.copy()
df_encoded['Wettercode'] = encoded_df[:, 0]

# Select only the columns needed for KNN imputation
df_knn = df_encoded[columns_for_knn + ['Wettercode']]

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=2)
imputed_df_knn = knn_imputer.fit_transform(df_knn)

# Separate the imputed 'Wettercode' values from the rest of the DataFrame
imputed_wettercodes = imputed_df_knn[:, -1]  # Extract the last column, which is 'Wettercode'

# Decode the imputed values back to the original categorical form
decoded_wettercodes = ordinal_encoder.inverse_transform(imputed_wettercodes.reshape(-1, 1))

# Replace the original 'Wettercode' column with the decoded imputed values
df_imp = merged_df.copy()
df_imp['Wettercode'] = decoded_wettercodes.flatten()

print(df_imp)


InvalidIndexError: (slice(None, None, None), 0)

In [19]:
print("\n merged_df:", merged_df['Wettercode'].describe())
print("\n df_imp:", df_imp['Wettercode'].describe())



 merged_df: count    7009.000000
mean       36.900128
std        27.095884
min         0.000000
25%        10.000000
50%        28.000000
75%        61.000000
max        95.000000
Name: Wettercode, dtype: float64

 df_imp: count    9334.000000
mean       27.708699
std        28.390122
min         0.000000
25%         0.000000
50%        20.000000
75%        61.000000
max        95.000000
Name: Wettercode, dtype: float64


In [20]:
# Print the frequency of each value of the weather codes (which ones occure?)
value_counts = df_imp['Wettercode'].value_counts().sort_index()
print(value_counts)

Wettercode
0.0     3140
3.0        5
5.0      562
10.0     742
17.0       5
20.0     244
21.0    1013
22.0      88
28.0      63
45.0      96
49.0      48
53.0      31
61.0    2271
63.0     535
65.0     111
68.0      12
69.0      27
71.0      81
73.0      48
75.0      25
77.0      45
79.0       5
95.0     137
Name: count, dtype: int64


## Ergebnis speichern:

In [11]:
# Save the updated data to new CSV files
df_imp.to_csv('../sourcedata/cleaned_data/6.2_missing_value_imputation-2.csv', sep=',', index=False)